In [1]:
print("OK")

OK


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
PINECONE_API_KEY = "pcsk_4jmtUP_Q3DBbnPppz5Hjk1y4khGFv5JDSb9wkUb6pWWUuQVgfNB91fAdZHnXcTaJyecjsd"
PINECONE_ENV = "us-east-1"

In [ ]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents